# Bias and Visualization

This tutorial will guide you through simple techniques to check if there is bias in grading and through interesting ways to look at the data.  

Let's start configuring this notebook to show large grapics.

In [1]:
options(repr.plot.width = 6, repr.plot.height = 6)

## Hipothesis Testing

Is the grade that woman receive in a class lower than the grade their male counterparts receive?  This is a very contentious question that commonly arise.  If you ask the instructors, they will tell you that difference in grades are the product of differences in performance and no bias is present.  Let's try to find a way to answer this question in a scientificly rigourous way.

First we will load the data.  You should be familiar with these dataset, as it was the one used during the academic data assignment.  It is provided by University of Michigan and it synthetically generated.  There are two files:  

* student.course.csv: Contains the knwon academic data with the relationship between student and courses
* student.record.csv: Contains information about the students (previous grades, sex, etc.)

Before we start we will load known libraries that we will use to manipulate the data.

In [5]:
library(dplyr)

Now we will load the student record dataset. 

NOTE: This are large datasets (similar to the ones that you will encounter in real institutions).  They take time to load.  Be patient. 

In [7]:
sr <- read.csv("student.record.csv")
colnames(sr)
head(sr)

[1] "MAJOR3_DESCR"          "MAJOR2_DESCR"          "MAJOR1_DESCR"         
 [4] "HSGPA"                 "LAST_ACT_ENGL_SCORE"   "LAST_ACT_MATH_SCORE"  
 [7] "LAST_ACT_READ_SCORE"   "LAST_ACT_SCIRE_SCORE"  "LAST_ACT_COMP_SCORE"  
[10] "LAST_SATI_VERB_SCORE"  "LAST_SATI_MATH_SCORE"  "LAST_SATI_TOTAL_SCORE"
[13] "SEX"                   "STDNT_GROUP1"          "STDNT_GROUP2"         
[16] "MAJOR1_DEPT"           "MAJOR2_DEPT"           "MAJOR3_DEPT"          
[19] "ANONID"                "ADMIT_TERM"            "MAJOR1_TERM"          
[22] "MAJOR2_TERM"           "MAJOR3_TERM"

MAJOR3_DESCR,MAJOR2_DESCR,MAJOR1_DESCR,HSGPA,LAST_ACT_ENGL_SCORE,LAST_ACT_MATH_SCORE,LAST_ACT_READ_SCORE,LAST_ACT_SCIRE_SCORE,LAST_ACT_COMP_SCORE,LAST_SATI_VERB_SCORE,...,STDNT_GROUP1,STDNT_GROUP2,MAJOR1_DEPT,MAJOR2_DEPT,MAJOR3_DEPT,ANONID,ADMIT_TERM,MAJOR1_TERM,MAJOR2_TERM,MAJOR3_TERM
<fct>,<fct>,<fct>,<dbl>,<int>,<int>,<int>,<int>,<int>,<int>,...,<fct>,<fct>,<fct>,<fct>,<fct>,<int>,<int>,<int>,<int>,<int>
NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,NA,NA,NA,NA,NA,1,NA,NA,NA,NA
NA,NA,Asian Studies BA,3.8,25,22,29,28,26,NA,...,NA,NA,Asian Languages And Cultures,NA,NA,2,110,123,NA,NA
NA,NA,Psychology BA,3.3,NA,NA,NA,NA,NA,NA,...,E,NA,Psychology Department,NA,NA,4,63,79,NA,NA
NA,NA,Civil Engineering BSE,3.9,33,32,28,30,31,NA,...,NA,NA,Civil & Environmental Engr,NA,NA,6,106,125,NA,NA
NA,NA,NA,3.8,24,22,18,18,21,610,...,C,NA,NA,NA,NA,7,83,NA,NA,NA
NA,NA,NA,3.9,31,22,34,22,27,NA,...,NA,NA,NA,NA,NA,8,126,NA,NA,NA


<font color="darkblue">**Explanation:**
<ul>
<li>We load the data from a csv file.
<li>We show the name of the columns that explain their content: HSGPA is the student High School GPA.  LAST_ACT_*_SCORE contain the score of the ACT test for a given topic.  SEX contains the gender of the student.  MAJOR*_DESCR contains the description of the final major of the students, while MAJOR*_DEPT indicates the department where that major exist.  The table also as an ID for the student (ANONID) and the term in which the student was accepted.
<li>As we can see in the table, not all values are present for all students.
</ul>
</font>

Now we load the data about the student course relationships.

NOTE: This dataset is even larger than the student data (to be expected).  Please be patient while the data is loaded into memory.

In [10]:
sc <- read.csv("student.course.csv")
head(sc)

ANONID,SUBJECT,CATALOG_NBR,GRD_PTS_PER_UNIT,GPAO,DIV,ANON_INSTR_ID,TERM
<int>,<fct>,<int>,<dbl>,<dbl>,<fct>,<int>,<int>
26,ACC,272,2.0,3.343636,P,2920,79
114,ACC,272,2.0,2.817857,P,2920,83
121,ACC,272,4.0,4.000000,P,201,111
125,ACC,272,1.3,3.639063,P,3360,84
180,ACC,272,3.0,3.846154,P,2920,107
207,ACC,272,2.0,2.715686,P,2920,93


<font color="darkblue">**Explanation:**
<ul>
<li>Again, we load the data from a csv file.
<li>This dataset has less columns: the ID of the student (ANONID), the subject of the course (ACC, PHYSICS, etc), the catalog number (CATALOG_NBR).  The combination of Subject and Catalog Numbers is what defines a unique course.  GRD_PTS_PER_UNIT has the grade that the student got in the course.  GPAO is the average grade obtained by the student in all the other courses. DIV has information about the type of course (Humanities, Sciences, etc.).  ANON_INSTR_ID has information about the id of the instructor.  Finally TERM is the semester in which the student took the course.
<li>This file does not have empty cells.
</ul>
</font>

Once the two datasets are loaded, we combined them into just one dataset using the ID of the student (ANONID) as the identificator for common rows.  

To combine the datasets we use the function merge.

In [9]:
data <- merge(sc,sr,by='ANONID',all.x=TRUE)
head(data)

ANONID,SUBJECT,CATALOG_NBR,GRD_PTS_PER_UNIT,GPAO,DIV,ANON_INSTR_ID,TERM,MAJOR3_DESCR,MAJOR2_DESCR,...,SEX,STDNT_GROUP1,STDNT_GROUP2,MAJOR1_DEPT,MAJOR2_DEPT,MAJOR3_DEPT,ADMIT_TERM,MAJOR1_TERM,MAJOR2_TERM,MAJOR3_TERM
<int>,<fct>,<int>,<dbl>,<dbl>,<fct>,<int>,<int>,<fct>,<fct>,...,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<int>,<int>,<int>,<int>
1,MATH,425,2.7,3.190000,S,1719,130,NA,NA,...,F,NA,NA,NA,NA,NA,NA,NA,NA,NA
2,POLSCI,160,3.0,3.566667,SS,3477,110,NA,NA,...,F,NA,NA,Asian Languages And Cultures,NA,NA,110,123,NA,NA
2,UC,280,4.0,3.440000,O,3604,114,NA,NA,...,F,NA,NA,Asian Languages And Cultures,NA,NA,110,123,NA,NA
2,UC,280,4.0,3.425926,O,1830,115,NA,NA,...,F,NA,NA,Asian Languages And Cultures,NA,NA,110,123,NA,NA
4,MATH,115,2.7,3.361538,S,3604,69,NA,NA,...,M,E,NA,Psychology Department,NA,NA,63,79,NA,NA
4,STATS,350,2.3,2.827500,S,3604,74,NA,NA,...,M,E,NA,Psychology Department,NA,NA,63,79,NA,NA


<font color="darkblue">**Explanation:**
<ul>
<li>We merge the sc and sr dataframe, using ANONID as the common information that link the two dataframes.  
<li>The all.x=TRUE parameter tell R that extra rows should be added to the output, one for each row in the first dataset that has no matching row in second dataset. If it is set to FALSE (or not set), only rows with data from both the first and second datasets are included in the output.
<li>As we can see in the table, this new dataset contains a comination of the two previous dataset.
</ul>
</font>

Once we have the data in just one dataset, we proceed to clean it.  One problem that the dataset has because it was syntetically generated is that the values of the GPA go beyond 4 and lower than 1 (the theoretical limits).  We filter the data, so no strange GPA is considered.

In [25]:
min_gpao <- 1
max_gpao <- 4
data = data %>% 
  filter(GPAO<=max_gpao) %>%
  filter(GPAO>=min_gpao) %>%
  filter(GRD_PTS_PER_UNIT<=max_gpao) %>%
  filter(GRD_PTS_PER_UNIT>=min_gpao)

<font color="darkblue">**Explanation:**
<ul>
<li>First we define two variables min_gpao and max_gpao with the theoretical limits of gpa.  
<li>Then we use the filter function to only consider students that have a GPAO higher or equal to 1 and lower or equal to 4.
</ul>
</font>

The unit of this analysis will be a course.  We select "PHYSICS 135" course. 

In [26]:
course_id=135
course_name="PHYSICS"

data_course =
  data %>%
  filter(SUBJECT==course_name & CATALOG_NBR==course_id)

head(data_course)

ANONID,SUBJECT,CATALOG_NBR,GRD_PTS_PER_UNIT,GPAO,DIV,ANON_INSTR_ID,TERM,MAJOR3_DESCR,MAJOR2_DESCR,...,SEX,STDNT_GROUP1,STDNT_GROUP2,MAJOR1_DEPT,MAJOR2_DEPT,MAJOR3_DEPT,ADMIT_TERM,MAJOR1_TERM,MAJOR2_TERM,MAJOR3_TERM
<int>,<fct>,<int>,<dbl>,<dbl>,<fct>,<int>,<int>,<fct>,<fct>,...,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<int>,<int>,<int>,<int>
128,PHYSICS,135,1.7,2.565753,S,2504,122,NA,NA,...,M,NA,NA,Microbiology And Immunology,NA,NA,118,131,NA,NA
136,PHYSICS,135,3.0,2.800000,S,970,126,NA,NA,...,F,NA,NA,NA,NA,NA,122,NA,NA,NA
139,PHYSICS,135,2.3,2.740000,S,970,106,NA,NA,...,M,NA,NA,Economics Department,NA,NA,102,115,NA,NA
144,PHYSICS,135,4.0,3.973333,S,1655,130,NA,NA,...,M,NA,G,NA,NA,NA,126,NA,NA,NA
210,PHYSICS,135,3.3,3.045946,S,2504,126,NA,NA,...,M,NA,NA,"Molecular, Cellular, and Developmental Biology",NA,NA,118,131,NA,NA
210,PHYSICS,135,2.7,3.431481,S,2647,123,NA,NA,...,M,NA,NA,"Molecular, Cellular, and Developmental Biology",NA,NA,118,131,NA,NA


<font color="darkblue">**Explanation:**
<ul>
<li>First we define two variables to contain the subject and id of the course. 
<li>We use those variables to filter only those rows in which the SUBJECT is "PHYSICS" and the CATALOG_NBR=135
</ul>
</font>

As we learned in a previous tutorial, the best way to eliminte the influece of the student performance from the grading behaviour in a class is to substract the GPA of the student from the grade that the student got into the course.  We call this quantity the Penalty.  That is how higher or lower the grade of the students is of the expected grade that they will obtain in an "average" course.

In [27]:
data_course$Penalty<-data_course$GRD_PTS_PER_UNIT-data_course$GPAO

Once we calculate the penalty that the students, we can visualize it to see what type of course PHYSICS 135 is.

In [28]:
plot(density(data_course$Penalty),main="Distribution of Penalty in PHYSICS 135")

We can see in the graph that while the Penalty seems to peak around 0, there is a non small proportion of students with a negative penalty.  That means that the course was difficult (lower grade than expected).

But we are not interested in knowing if the course is difficult or not.  We are interested in knowing if females are graded more strictly than males, when the individual performance (measured as GPA) is taken out of the equation.  For that we divide the total poulation of the course by gender.

In [29]:
fem_data <- data_course %>%
  filter(SEX=='F')

mal_data <- data_course %>%
  filter(SEX=='M')

Now for reference, we plot the density distribution of the Penalty for males (red) and females (black).

In [30]:
plot(density(mal_data$Penalty),col='red', main="Distribution of Penalty")
lines(density(fem_data$Penalty))

It is difficult to see much difference between these two distribution.  It seems that there are more negative penalties for female students, but we will have a hard time convincing anyone that this difference is real, specially a professor that will be very defensive about its grading policies.

Let's try another visualization to see if we have a better luck.

In [31]:
boxplot(fem_data$Penalty,mal_data$Penalty,main="Penalty Comparison")

Again, these boxplots seem to indicate that there is a difference, but how large is that difference and it is just the product of random variations in the population?  

While visualizations are good are providing an insights into the data, they cannot be used as proof of anything.  We need to use statistical tests that could provide a more solid ground to draw conclusions out of the data.

The most common test to prove differences between populations is the <a href="https://en.wikipedia.org/wiki/Student%27s_t-test">T-test</a>.  We will apply it to see what is the result.

In [32]:
t.test(fem_data$Penalty, mal_data$Penalty)


	Welch Two Sample t-test

data:  fem_data$Penalty and mal_data$Penalty
t = -9.0794, df = 3786.5, p-value < 2.2e-16
alternative hypothesis: true difference in means is not equal to 0
95 percent confidence interval:
 -0.1936771 -0.1248872
sample estimates:
  mean of x   mean of y 
-0.19124101 -0.03195884 


To fully understand this result, we need knowledge about statistics.  In short, the test says that there IS a difference between the penalty of both populations that cannot be explained by chance (p-value almost 0).  It also tell use that in average women have a penalty of -0.32 points (in a 1-4 scale) compared with a penalty of -0.13 for men taking the same class.  This seems to be a very good prove that there is a non-explained difference on grades received by men and women in this course.

Not so fast.  Statistics tests are powerful tools, but as any tool, you need to know when it is possible to use it and which are their limitations.  T-tests are only valid if the underlying distribution of the compared populations approach a normal distribution.  The density graphs that we created before already indicate that the distributions are skewed.  To test if we can conduct a T-test we should test the normality of the distributions.  We can do it through the <a href="https://en.wikipedia.org/wiki/Shapiro%E2%80%93Wilk_test">Shapiro-Wilk</a> test.

In [33]:
shapiro.test(fem_data$Penalty)
shapiro.test(mal_data$Penalty)


	Shapiro-Wilk normality test

data:  fem_data$Penalty
W = 0.95359, p-value < 2.2e-16



	Shapiro-Wilk normality test

data:  mal_data$Penalty
W = 0.95234, p-value < 2.2e-16


The Shapiro-Wilk test confirms our suspisions, we should reject the hipothesis that the distributions are normal.  We should find another test that give us information about the dissimilarity of the populations but that not require that the underlying distributions are normal.  Fortunately, a non-parametric test such as <a href="https://en.wikipedia.org/wiki/Kolmogorov%E2%80%93Smirnov_test">Kolmogorov-Smirnov</a>. 

In [34]:
ks.test(fem_data$Penalty,mal_data$Penalty)

Warning message in ks.test(fem_data$Penalty, mal_data$Penalty):
“p-value will be approximate in the presence of ties”


	Two-sample Kolmogorov-Smirnov test

data:  fem_data$Penalty and mal_data$Penalty
D = 0.15369, p-value < 2.2e-16
alternative hypothesis: two-sided


The result now is strong.  Men receive less penalty in their grades that women, even when their GPA is taken our of the equation.  This is a clear indicator that there seems to be some type of bias in the grading that is not the product of chance.  While it is not proof that the teacher is discriminating against women, it should be the start of a deeper analysis of the current situation.

One possible next step is to see the differences for students at different ranges of GPA.  For this we divide both men and women into 15 bins according to their GPA and compare the difference in average grade.

In [35]:
nbins=15
binsize  <- (max_gpao-min_gpao)/nbins
data_course$bin<-floor((data_course$GPAO-min_gpao)/binsize) + 1

<font color="darkblue">**Explanation:**
<ul>
<li>We divide the range of gpa (from 1 to 4) into 15 bins to obtain the length of each bin.
<li>Then we calculate the bean for each one of the students and store this information in the bin column.
</ul>
</font>

Now we calcualte the mean GPA, mean grade and standard error in grade for each bin, separately for woman, men and all the students together.

In [36]:
fem.binned = data_course %>%
  filter(SEX=='F') %>%
  group_by(bin) %>%
  summarize(
    meanGpa = mean(GPAO),
    meanGrade = mean(GRD_PTS_PER_UNIT),
    seGrade = sd(GRD_PTS_PER_UNIT/sqrt(n()))
  )

mal.binned <- data_course %>%
  filter(SEX=='M') %>%
  group_by(bin) %>%
  summarize(
    meanGpa = mean(GPAO),
    meanGrade = mean(GRD_PTS_PER_UNIT),
    seGrade = sd(GRD_PTS_PER_UNIT/sqrt(n()))
  )

all.binned <- data_course %>%
  group_by(bin) %>%
  summarize(
    meanGpa = mean(GPAO),
    meanGrade = mean(GRD_PTS_PER_UNIT),
    seGrade = sd(GRD_PTS_PER_UNIT/sqrt(n()))
  )

<font color="darkblue">**Explanation:**
<ul>
<li>For each group of interest we group the students by bins, then we calculate the mean gpa, the mean grade in the class and the standard error in grade.
</ul>
</font>

Now we plot the the mean GPA vs. the mean Grade for each one male and female bins.  

In [38]:
plot(mal.binned$meanGpa,mal.binned$meanGrade,pch=19,col='red',xlab='GPAO',ylab='<Grade>',main="Comparison between Penalty",xlim=c(1,4),ylim=c(1,4))
arrows(mal.binned$meanGpa,mal.binned$meanGrade-mal.binned$seGrade,mal.binned$meanGpa,mal.binned$meanGrade+mal.binned$seGrade,code=0,col='red')

points(fem.binned$meanGpa,fem.binned$meanGrade,pch=19)
arrows(fem.binned$meanGpa,fem.binned$meanGrade-fem.binned$seGrade,fem.binned$meanGpa,fem.binned$meanGrade+fem.binned$seGrade,code=0)

lines(c(1,4),c(1,4)) 

<font color="darkblue">**Explanation:**
<ul>
<li>First we put the male points (red).
<li>The second line add the error bars (standard error) for each male point.
<li>Then we do the same for the female poinsts (black).
<li>Finally we create a line of 0 penalty for comparison.
</ul>
</font>

Now we can see that the difference is not equal across different GPAs.  Top women performers do not seem to feel any bias, however, below 3.5, it seems that men always receive a lower penalty than women. 

This type of visualizations are conversation-starters for sensemaking around grading policies.

## Visualization

“Visualization is really about external cognition, that is, how resources outside the mind can be used to boost the cognitive capabilities of the mind.” — Stuart Card

We use visualization to summarize large amount of data and relationships between that data. Here we will cover different types of less frequent types of visualizations to represent different types of analysis.

### Distribution 

We have already explored the use of <a href="https://www.r-graph-gallery.com/boxplot.html">boxplots</a>, <a href="https://www.r-graph-gallery.com/density-plot.html">density plots</a> and <a href="https://www.r-graph-gallery.com/histogram.html">histograms</a> to show visualize the distribution of data.  

We will see to similar plots, the Violins and the Ridges.

#### The Violin Plots

Violin plot allows to visualize the distribution of a numeric variable for one or several groups. Each ‘violin’ represents a group or a variable. The shape represents the density estimate of the variable: the more data points in a specific range, the larger the violin is for that range. It is really close to a boxplot, but allows a deeper understanding of the distribution.

Here we will create a violin plot to compare the distribution of penalty values for both males and females in the dataset.

In [39]:
library(ggplot2)

ggplot(data_course, aes(x=SEX, y=Penalty, fill=SEX)) +
  geom_violin()

<font color="darkblue">**Explanation:**
<ul>
<li>We will use a very common library for visualizations in R, ggplot.
<li>Ggplot is a graphical language, it means that we construct the graph from simple directives added to each other.
<li>We first specify which dataset we will be using, in this case data_course.
<li>Then we specify the aesthetics (aes), that is the components of the graph.  The x axis will represent the different genders (SEX), the y axis will represent the values of penalty (Penalty) and the color (fill) will be assigned by the gender (SEX).  
<li>In the next line we specify that the geometry of a violin (geom_violin) will be used to represent the data.
</ul>
</font>

#### Ridgeline

A Ridgeline plot (sometimes called Joyplot) shows the distribution of a numeric value for several groups. Distribution can be represented using histograms or density plots, all aligned to the same horizontal scale and presented with a slight overlap.

Here we will create a plot of the different penalty distributions for different courses.  

First we need to install and import the libraries that will allow us to create such visualization.

In [43]:
install.packages("ggridges")
install.packages("hrbrthemes")

library(ggridges)
library(ggplot2)
library(viridis)
library(hrbrthemes)

Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)


Next, we need to create the data to visualize.  In this case we will create a list of courses at different levels and calculate the Penalty values for all their students.

In [45]:
courses_id=c(100,115,124,135,215,240,280,300,380)

data_courses =
  data %>%
  filter(CATALOG_NBR %in% courses_id)

data_courses$Penalty <- data_courses$GRD_PTS_PER_UNIT-data_courses$GPAO
data_courses$course <-as.factor(data_courses$CATALOG_NBR)

Finally, we send that data to ggplot to be drawn

In [46]:
ggplot(data_courses, aes(x = Penalty, y = course, fill = course)) +
  geom_density_ridges() +
  theme_ridges() + 
  theme(legend.position = "none")

Picking joint bandwidth of 0.0585


<font color="darkblue">**Explanation:**
<ul>
<li>We set the x axis for the representation of Penalty and the y axis will represent the course.  The color (fill) will use based on different courses (each course will receive its own color)
<li>The geometry to use is the density ridges (geom_density_ridges)
<li>It will use the theme_ridges to assign the colors and decorations
<li>It will not create a legend, as it is obvious from the graph.
</ul>
</font>

We can also base the color of the graphs on the values of penalty.  Instead of having one color per course, we will have a degrade shared by all the courses.

In [221]:
ggplot(data_courses, aes(x = Penalty, y = course, fill = ..x..)) +
  geom_density_ridges_gradient(scale = 3, rel_min_height = 0.01) +
  scale_fill_viridis(name = "Penalty", option = "C") +
  labs(title = 'Penalty Distribution in Different Courses') +
  theme_ipsum() +
    theme(
      legend.position="none",
      panel.spacing = unit(0.1, "lines"),
      strip.text.x = element_text(size = 8)
    )

Picking joint bandwidth of 0.0621


<font color="darkblue">**Explanation:**
<ul>
<li>We set the x axis for the representation of Penalty and the y axis will represent the course.  The color (fill) will be relative to the x axis.
<li>The geometry to use is the density ridges with gradient (geom_density_ridges_gradient)
<li>We use a scale to create a degrade in the fill (scale_fill_viridis)
<li>The rest are decorations for the graph.
</ul>
</font>

## Comparison

We have used the <a href="https://www.r-graph-gallery.com/scatterplot.html">scatter plot</a> to show comparison between two numerical variables.  If more variables are compared at the same time a good alternative is the heatmap and the 2-d density plot.

#### Heatmap
A heatmap is a graphical representation of data where the individual values contained in a matrix are represented as colors. It is a bit like looking a data table from above.

It is really useful to display a general view of numerical data, not to extract specific data point.

In this example we will visualize the grades obtained by students in different tests before they enter the university.  

First, we create the data as a numerical matrix.

In [47]:
student_data<-sr %>%
   select(HSGPA,LAST_ACT_ENGL_SCORE,LAST_ACT_MATH_SCORE,LAST_ACT_READ_SCORE,LAST_ACT_SCIRE_SCORE,LAST_ACT_COMP_SCORE)
rownames(student_data)<-sr$ANONID
colnames(student_data)<-c("HSGPA","ENG","MATH","READ","SCI","COMP")
head(student_data)

,HSGPA,ENG,MATH,READ,SCI,COMP
,<dbl>,<int>,<int>,<int>,<int>,<int>
1,NA,NA,NA,NA,NA,NA
2,3.8,25,22,29,28,26
4,3.3,NA,NA,NA,NA,NA
6,3.9,33,32,28,30,31
7,3.8,24,22,18,18,21
8,3.9,31,22,34,22,27


There are a lot of student in this table, some with empty fields.  We filter this table to just have 100 complete cases.

In [49]:
student_data<-student_data[complete.cases(student_data), ]
student_data<-student_data[1:100,]

Now we convert that table into a matrix (think a only numerical excel table) and then compute the heatmap.

In [51]:
matrix_data <- as.matrix(student_data)

# Default Heatmap
heatmap(matrix_data,scale="column",col=heat.colors(256))

<font color="darkblue">**Explanation:**
<ul>
<li>The heatmap function is used to create the dataset.
<li>To have comparable colors, we normalize the data with the scale option, based on columns.
<li>We use a range of colors to represent low values as yellow and high values as red.
<li>To be useful, the values are clustered based on their values.  
<li>It is easy to see that students group by their SAT scores more than by theri High School GPA.
</ul>
</font>

#### 2-D Density Plot

A 2d density plot shows the distribution of values in a dataset across the range of two quantitative variables.

It counts the number of observations within a particular area of the 2D space and represent it using square, hexagon, or kernel density estimation.

We will calculate the 2-d density plot of the Penalty vs. the GPA of female students. 

NOTE:  This graph is not showing in this notebook.  To see it, run the code into your R-Studio.  

In [52]:
ggplot(fem_data, aes(x=Penalty, y=GPAO) ) +
  geom_bin2d(bins=5) 

Here is what you should see: 
![2D-Density Plot](2ddensity.png)

### Ranking

When we want to see how different elements compare to one another in different dimensions we can use <a href="https://www.r-graph-gallery.com/barplot.html">barplots</a>, <a href="https://www.r-graph-gallery.com/spider-or-radar-chart.html">radar plots</a> or <a href="https://www.r-graph-gallery.com/wordcloud.html">wordclouds</a>.  

We can use another visualization technique that is called parallel coordinates.  

#### Parallel coordinates

Parallel plot or parallel coordinates plot allows to compare the feature of several individual observations (series) on a set of numeric variables. Each vertical bar represents a variable and often has its own scale. (The units can even be different). Values are then plotted as series of lines connected across each axis.

For this example, we will cluster students based on their scores in SAT test (using the matrix that we created previously).

In [53]:
clusters<-kmeans(matrix_data,4)
student_data$cluster<-as.factor(clusters$cluster)

Now we load the required libraries.

In [54]:
library(hrbrthemes)
library(GGally)
library(viridis)


Attaching package: ‘GGally’

The following object is masked from ‘package:dplyr’:

    nasa



Finally we call the ggparcoord function to plot the data.

In [55]:
ggparcoord(student_data,
    columns = 1:6, groupColumn = 7, order = "anyClass",
    scale="uniminmax",
    showPoints = TRUE, 
    title = "Standardize to Min = 0 and Max = 1",
    alphaLines = 0.3
    ) + 
  scale_color_viridis(discrete=TRUE) +
  theme_ipsum()+
  theme(
    legend.position="none",
    plot.title = element_text(size=13)
  ) +
  xlab("")

### Components

When we want to present how a population is composed, we have several alternatives such as the <a href="https://www.r-graph-gallery.com/pie-plot.html">pie chart</a>.  We will explore an alternative when the data is subdivided in categories, the treemap.

#### Treemap

A Treemap displays hierarchical data as a set of nested rectangles. Each group is represented by a rectangle, which area is proportional to its value. Using color schemes, it is possible to represent several dimensions: groups, subgroups etc.

Treemaps have the advantage to make efficient use of space, what makes them useful to represent a big amount of data.

If you have many levels of hierarchy (>2), it is advised to build an interactive figure: clicking on a upper level of organization will reveal the next level and so on.

We first will represent all the courses that have been taken before PHYSICS 135.  For that we compare the term in which PHYSICS 135 was taken by an student from the term in which other courses were taken by the same student.

In [56]:
students_physics <- unique(data_course$ANONID)
term_physics <- data_course[,c("ANONID","TERM")]
names(term_physics) <- c("ANONID","TAKEN")


related_courses<-sc[sc$ANONID %in% students_physics,c( "ANONID","SUBJECT","CATALOG_NBR","GRD_PTS_PER_UNIT","GPAO","DIV","TERM")]
related_courses <- merge(related_courses,term_physics,by='ANONID',all.x=TRUE)

SEQTERM    <- mat.or.vec(length(related_courses$ANONID),1)
related_courses <- data.frame(related_courses,SEQTERM)
  
eb   <- which(related_courses$TERM < related_courses$TAKEN)
ea   <- which(related_courses$TERM > related_courses$TAKEN)
es   <- which(related_courses$TERM == related_courses$TAKEN)
lenb <- length(eb)
lena <- length(ea)
lens <- length(es)
  
related_courses$SEQTERM[eb] <- 1
related_courses$SEQTERM[ea] <- 2

related_courses$course <-paste(related_courses$SUBJECT, related_courses$CATALOG_NBR,sep = " ")

courses_before <- related_courses %>%
    filter(SEQTERM==1) %>%
    group_by(course) %>%
    summarize(
        students=n()
    )



<font color="darkblue">**Explanation:**
<ul>
<li>In the first 3 lines of the code, we select all the students that have taken PHYSICS 135 course.  Then we attach the term in which they have taken this course in the TAKEN column.  Finally we create a dataset with only the ID of the student and the TAKEN column.
<li>Then we select all the courses that have been taken by the selected students. We then add the TAKEN column to that dataset using the merge command.
<li>We create a SEQTERM vector that will contain 1 if the course was taken before PHYSICS 135, 2 if was taken after and 0 if it was taken durin the same semester.
<li>Finally we create a dataset (courses_before) with the name of the courses that were taken before and the number of student who took them.
</ul>
</font>

We plot the data, using the id of the course as name of the squares and the number of students to represent their size.  

First we need to install and load the treemap library.

In [59]:
install.packages("treemap")
library(treemap)
treemap(courses_before,
            index="course",
            vSize="students",
            type="index"
            )

Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)


While this graph is interesting, it has very little advantage over a pie chart.  However, the real power of a treemap is when we add categories to the data.  In this case we will categorize the courses based on their School (Division)

In [60]:
division <- mat.or.vec(length(related_courses$course),1)

e <- related_courses$DIV == 'S'
division[e]   <- 'NS'
e <- related_courses$DIV == 'H'
division[e]   <- 'Hu'
e <- related_courses$DIV == 'SS'
division[e]   <- 'SS'
e <- related_courses$DIV == 'O'
division[e]   <- 'Oth'
e <- related_courses$DIV == 'P'
division[e]   <- 'Pro'
e <- related_courses$DIV == 'E'
division[e]   <- 'Eng'

related_courses$division <- as.factor(division)

courses_before_grouped <- related_courses %>%
    filter(SEQTERM==1) %>%
    group_by(course,division) %>%
    summarize(
        students=n()
    )

Now when we plot the treemap we specify that we want to group the course boxes by their division.

In [61]:
treemap(courses_before_grouped,
            index=c("division","course"),
            vSize="students",
            type="index"
            ) 

### Flow 

Representing flow is usually the task of <a href="https://www.r-graph-gallery.com/network.html">networks</a>.  However there is a special type of network, called the Sankey Network.

#### Sankey Network
A Sankey Diagram is a visualisation technique that allows to display flows. Several entities (nodes) are represented by rectangles or text. Their links are represented with arrows or arcs that have a width proportional to the importance of the flow.

We will represent the flow between different physics courses using a Sankey diagram.

First we need to create the data.  We select only the PHYSICS courses.


In [73]:
data = data %>% 
  filter(GPAO<=max_gpao) %>%
  filter(GPAO>=min_gpao) 

data_physics =
  data %>%
  filter(SUBJECT=="PHYSICS")

Next, we create a function to get all the previous courses of a previous course, based on the code used for the heatmap.

In [74]:
get_previous <-function(x)
{
    course_data<-data_physics %>% filter(CATALOG_NBR==x)
    students_course <- unique(course_data$ANONID)
    term_physics <- course_data[,c("ANONID","TERM")]
    names(term_physics) <- c("ANONID","TAKEN")
    related_courses<-data_physics[data_physics$ANONID %in% students_course,c( "ANONID","SUBJECT","CATALOG_NBR","GRD_PTS_PER_UNIT","GPAO","DIV","TERM")]
    related_courses <- merge(related_courses,term_physics,by='ANONID',all.x=TRUE)
    previous<-related_courses %>%
       filter(TERM<TAKEN) %>%
       group_by(CATALOG_NBR) %>%
       summarize(
          count=n()
       )
    previous$Target<-x
    colnames(previous)<-c("Source","Weight","Target")
    previous[,c("Source","Target","Weight")]
}

We then apply the function to all the PHYSICS courses to create the links dataset. This dataset contains the links between courses.

In [75]:
links<-lapply(unique(data_physics$CATALOG_NBR),get_previous)
links<-bind_rows(links)
links<-as.data.frame(links)

We use networkD3 library to visualize the data. 

The node dataset is created from the names of the linked courses.

In [77]:
install.packages("networkD3")
library(networkD3)
 
nodes <- data.frame(
  name=c(as.character(links$Source), 
  as.character(links$Target)) %>% unique()
)
 
links$IDsource <- match(links$Source, nodes$name)-1 
links$IDtarget <- match(links$Target, nodes$name)-1
 


Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)


We send the data to the sankeyNetwork function

In [79]:
# Make the Network
p <- sankeyNetwork(Links = links, Nodes = nodes,
              Source = "IDsource", Target = "IDtarget",
              Value = "Weight", NodeID = "name", 
              sinksRight=FALSE)
p

HTML widgets cannot be represented in plain text (need html)

## Creating a Dashboard

Now we will create a dashboard to showcase what we did during this tutorial.

This dashboard will have two sections:  
* One will let us explore the different courses and test if there is grade bias between males and females.
* The second one will be as gallery of the rest of visualizations

You should create this dashboard in R-Studio

The dashboard file could be downloaded <a href="https://notebooks.azure.com/xavierochoa/projects/bla2/html/Session%2005/Tutorial/Dash.Rmd">here</a>.